In [70]:
import pandas as pd
import numpy as np
data=pd.read_csv('wine_data.csv')
data.head(5)

,Y,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines\t,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [71]:
y=data.Y
x=data.drop('Y',1)
#print(x.head(5))
#print(y.head(5))

C:\Users\khwan\AppData\Local\Temp/ipykernel_12328/2118091226.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x=data.drop('Y',1)


In [72]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,random_state=0)
type(x_train)

pandas.core.frame.DataFrame

In [73]:
print(x_train.shape)
print(x_test.shape)

(133, 13)
(45, 13)


In [74]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
x_train = Scaler_X.fit_transform(x_train)
x_test = Scaler_X.transform(x_test)
type(x_train)
type(y_train)

pandas.core.series.Series

In [75]:
#x_train = x_train.to_numpy()
#y_train = y_train.to_numpy()
x_train=x_train.astype(np.float32)
#y_train=y_train.astype(int)
input_dim=13
output_dim=4
type(y_train)

pandas.core.series.Series

In [76]:
import sys
print(f"Python{sys.version}\n")
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 

Python3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]



In [77]:
model=tf.keras.models.Sequential()
#input layer
model.add(tf.keras.Input(shape=(input_dim)))

#hidden layer
model.add(tf.keras.layers.Dense(20,activation='relu',name='Hidden1'))
model.add(tf.keras.layers.BatchNormalization(axis=-1,name='BN1'))
model.add(tf.keras.layers.Dense(20,activation='relu',name='Hidden2'))
model.add(tf.keras.layers.BatchNormalization(axis=-1,name='BN2'))
model.add(tf.keras.layers.Dense(20,activation='relu',name='Hidden3'))
model.add(tf.keras.layers.BatchNormalization(axis=-1,name='BN3'))
model.add(tf.keras.layers.Dropout(0.4))

#output layer
model.add(tf.keras.layers.Dense(output_dim,activation='softmax',name='output'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden1 (Dense)             (None, 20)                280       
                                                                 
 BN1 (BatchNormalization)    (None, 20)                80        
                                                                 
 Hidden2 (Dense)             (None, 20)                420       
                                                                 
 BN2 (BatchNormalization)    (None, 20)                80        
                                                                 
 Hidden3 (Dense)             (None, 20)                420       
                                                                 
 BN3 (BatchNormalization)    (None, 20)                80        
                                                                 
 dropout_3 (Dropout)         (None, 20)               

In [78]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])


In [79]:
display(y_train)
np.shape(y_train)

19     1
135    3
74     2
144    3
16     1
      ..
103    2
67     2
117    2
47     1
172    3
Name: Y, Length: 133, dtype: int64

(133,)

In [80]:
display(x_train)
np.shape(x_train)

array([[ 0.79996866,  0.63400364,  0.71783316, ...,  0.05445565,
         1.0713277 ,  0.3150045 ],
       [-0.4987598 ,  0.06171955, -0.610727  , ..., -0.9347358 ,
        -1.3924971 , -0.18138885],
       [-1.2979773 , -1.163326  , -0.24168251, ...,  0.18348062,
         0.7529683 ,  0.45068535],
       ...,
       [-0.72353977, -0.69834524, -0.64763147, ...,  0.48453888,
         0.5176592 , -1.33964   ],
       [ 1.1246508 , -0.63575166, -0.9059626 , ..., -0.16058595,
         1.0298026 ,  0.778305  ],
       [ 1.449333  ,  0.10642924,  0.42259756, ..., -1.4078273 ,
        -1.2125548 , -0.29721397]], dtype=float32)

(133, 13)

In [81]:
checkpoint_filepath = "bestmodel_epoch{epoch:02d}_valloss{val_loss:.2f}.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               monitor='Val_acc',
                                                               mode='max',
                                                               save_best_only=True)

In [84]:
history=model.fit(x_train,y_train,batch_size=100,epochs=10,verbose=1,validation_split=0.2,callbacks=[model_checkpoint_callback])

Epoch 1/10
2/2 [==============================] - 0s 77ms/step - loss: 0.9879 - acc: 0.5472 - val_loss: 1.2512 - val_acc: 0.5185
Epoch 2/10
2/2 [==============================] - 0s 41ms/step - loss: 0.9388 - acc: 0.5755 - val_loss: 1.2365 - val_acc: 0.5185
Epoch 3/10
2/2 [==============================] - 0s 85ms/step - loss: 0.9599 - acc: 0.6038 - val_loss: 1.2225 - val_acc: 0.5556
Epoch 4/10
2/2 [==============================] - 0s 75ms/step - loss: 1.0480 - acc: 0.5660 - val_loss: 1.2099 - val_acc: 0.5556
Epoch 5/10
2/2 [==============================] - 0s 118ms/step - loss: 0.9821 - acc: 0.5849 - val_loss: 1.1966 - val_acc: 0.6296
Epoch 6/10
2/2 [==============================] - 0s 73ms/step - loss: 0.8049 - acc: 0.6698 - val_loss: 1.1841 - val_acc: 0.6296
Epoch 7/10
2/2 [==============================] - 0s 84ms/step - loss: 0.7762 - acc: 0.7264 - val_loss: 1.1718 - val_acc: 0.6296
Epoch 8/10
2/2 [==============================] - 0s 93ms/step - loss: 0.9647 - acc: 0.6321 - va

In [ ]:
type(x_train)